## Deliverable 2. Create a Customer Travel Destinations Map.

In [25]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/tkoid/OneDrive/Desktop/GW Bootcamp/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kobryn,BY,2021-04-05 04:50:33,52.2138,24.3564,32.00,86,0,8.95,clear sky
1,1,Hermanus,ZA,2021-04-05 04:50:33,-34.4187,19.2345,63.00,91,99,4.00,overcast clouds
2,2,Mananjary,MG,2021-04-05 04:50:33,-21.2167,48.3333,78.21,81,13,6.58,light rain
3,3,Yellowknife,CA,2021-04-05 04:47:57,62.4560,-114.3525,12.20,62,75,6.91,broken clouds
4,4,Mwaya,TZ,2021-04-05 04:50:33,-9.5500,33.9500,73.72,83,84,0.31,light rain


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 90


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Avarua,CK,2021-04-05 04:48:23,-21.2078,-159.7750,80.60,89,75,5.75,light rain
8,8,Busselton,AU,2021-04-05 04:50:34,-33.6500,115.3333,82.00,55,2,1.99,clear sky
12,12,Hambantota,LK,2021-04-05 04:50:35,6.1241,81.1185,84.20,79,0,9.22,clear sky
27,27,Kavieng,PG,2021-04-05 04:50:39,-2.5744,150.7967,82.35,74,86,5.01,light rain
31,31,Kudahuvadhoo,MV,2021-04-05 04:50:40,2.6708,72.8944,83.95,73,50,10.56,scattered clouds
43,43,Hobyo,SO,2021-04-05 04:50:42,5.3505,48.5268,84.13,63,48,7.58,scattered clouds
46,46,Atuona,PF,2021-04-05 04:50:43,-9.8000,-139.0333,80.13,80,77,11.52,broken clouds
48,48,Yulara,AU,2021-04-05 04:50:44,-25.2406,130.9889,86.00,19,2,11.50,clear sky
54,54,Butaritari,KI,2021-04-05 04:50:45,3.0707,172.7902,82.33,79,81,12.82,light rain
71,71,Souillac,MU,2021-04-05 04:50:50,-20.5167,57.5167,82.40,74,40,14.97,scattered clouds


In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                106
City                   106
Country                106
Date                   106
Lat                    106
Lng                    106
Max Temp               106
Humidity               106
Cloudiness             106
Wind Speed             106
Current Description    106
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Avarua,CK,80.60,light rain,-21.2078,-159.7750,
8,Busselton,AU,82.00,clear sky,-33.6500,115.3333,
12,Hambantota,LK,84.20,clear sky,6.1241,81.1185,
27,Kavieng,PG,82.35,light rain,-2.5744,150.7967,
31,Kudahuvadhoo,MV,83.95,scattered clouds,2.6708,72.8944,
43,Hobyo,SO,84.13,scattered clouds,5.3505,48.5268,
46,Atuona,PF,80.13,broken clouds,-9.8000,-139.0333,
48,Yulara,AU,86.00,clear sky,-25.2406,130.9889,
54,Butaritari,KI,82.33,light rain,3.0707,172.7902,
71,Souillac,MU,82.40,scattered clouds,-20.5167,57.5167,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [38]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('',np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df
clean_hotel_df = hotel_df
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Avarua,CK,80.60,light rain,-21.2078,-159.7750,Paradise Inn
8,Busselton,AU,82.00,clear sky,-33.6500,115.3333,Observatory Guest House
12,Hambantota,LK,84.20,clear sky,6.1241,81.1185,Bungalow 63
27,Kavieng,PG,82.35,light rain,-2.5744,150.7967,Nusa Island Retreat
31,Kudahuvadhoo,MV,83.95,scattered clouds,2.6708,72.8944,Niyama Private Islands Maldives
...,...,...,...,...,...,...,...
640,Popondetta,PG,89.24,broken clouds,-8.7537,148.2534,Timos Transit House
648,Auki,SB,83.93,overcast clouds,-8.7676,160.7034,Auki Motel
674,Honiara,SB,83.34,moderate rain,-9.4333,159.9500,Heritage Park Hotel
675,Akyab,MM,87.37,clear sky,20.1500,92.9000,Hotel Memory


In [39]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [40]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [45]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure()

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))